In [ ]:
%load_ext autoreload
%autoreload 2

from tqdm import tqdm
from math import sqrt,erf
from multiprocessing import Pool
from itertools import product, repeat
from matplotlib import pyplot as plt
from matplotlib import animation
import matplotlib as mpl
import numpy as np
from varname import nameof
from itertools import product
from matplotlib.cm import get_cmap
import colorcet as cc
import subprocess
import os, shutil
from time import time
import functools
from scipy import special
import scipy
from IPython.display import display, clear_output
import wavefunctionRGlib as wfrg


plt.style.use('ggplot')
markers = ["o", "X", "P", "p", "*"]
cols = [p['color'] for p in plt.rcParams['axes.prop_cycle']]
plt.rc('text.latex', preamble=r'\usepackage{amsmath}\usepackage{braket}\usepackage{nicefrac}')
plt.rcParams.update({'font.size': 30,
                     'figure.figsize': (11,7),
                     'axes.facecolor': 'white',
                     'axes.edgecolor': 'lightgray',
                     "figure.autolayout": 'True',
                     'axes.xmargin': 0.03,
                     'axes.ymargin': 0.05,
                     'axes.grid': False,
                     'axes.linewidth': 5,
                     'lines.markersize': 15,
                     'text.usetex': True,
                     'lines.linewidth': 8,
                     "legend.frameon": True,
                     "legend.framealpha": 0.7,
                     "legend.handletextpad": 1,
                     "legend.edgecolor": "black",
                     "legend.handlelength": 1,
                     "legend.labelspacing": 0,
                     "legend.columnspacing": 1,
                     "legend.fontsize": 35,
                    })
linestyles = ["-", "--", ":"]
bbox = dict(boxstyle="round", facecolor="lightgray")

D0 = 1
deltaD = 0.001
plt.plot([], [])
plt.show()
clear_output()

In [ ]:
def get_RG_flow(J0_by_D0, Ub_by_J=0, plot=False):
    """ Returns the flow of couplings in the form of two ndarrays J and D.
    Each ndarray is in ascending order of the bandwidth. """
    J0 = D0 * J0_by_D0
    Ub = - Ub_by_J * J0
    omega = -D0/2
    
    ### initialise arrays with UV values
    D = [D0]
    J = [J0]
    
    ### apply URG transformations until bandwith vanishes
    ### or J reduces to zero.
    while D[-1] >= deltaD and J[-1] >= 0:
        
        ### URG equation
        deltaJ = - J[-1] * (J[-1] + 4 * Ub) / (omega - D[-1]/2 + J[-1]/4) * deltaD
        
        ### Check if denominator has changed sign, 
        ### then append renormalised values to array
        if (omega - (D[-1] - deltaD)/2 + (J[-1] + deltaJ)/4) * (omega - D0/2 + J0/4) > 0:
            D.append(D[-1] - deltaD)
            J.append(J[-1] + deltaJ)
        else:
            break
    
    ### plot values
    if plot:
        plt.plot(np.array(D)/D0, np.array(J)/J0, marker="o")
        plt.xlabel(r"$D/D_0$")
        plt.ylabel(r"$J/J_0$")
    return np.flip(D), np.flip(J)

for J0_by_D0 in np.linspace(0.4, 5, 100):
    D, J = get_RG_flow(J0_by_D0)
    plt.scatter(J0_by_D0, D[0] / deltaD, color=cols[0])
    # plt.scatter(J0_by_D0, J[0], color=cols[1])
plt.xscale("log")
plt.show()

In [ ]:
def KondoMERG(J0_by_D0, num_entangled, both_sectors=False):
    D, J = get_RG_flow(J0_by_D0)
    omega = -D0/2
    time_list = []
    for num_IOMs in range(2, 5, 2):
        t = time()
        Ek = np.linspace(D[0] - deltaD * num_entangled, D[0], num_entangled)
        if both_sectors:
            alpha_arr = np.repeat([Ji / (2 * (omega - Di/2 + Ji/4)) for Ji, Di in zip(J, D)], 2)[:num_IOMs]
            IOMconfigs = np.tile([1, 0], num_IOMs // 2)
        else:
            alpha_arr = [Ji / (2 * (omega - Di/2 + Ji/4)) for Ji, Di in zip(J, D)][:num_IOMs]
            IOMconfigs = [0] * num_IOMs
        # print (Ek)
        init_couplings = [Ek, J[0], 0]
        decomposition_arr = wfrg.getWavefunctionRG(init_couplings, alpha_arr, num_entangled, num_IOMs, 
                                                   IOMconfigs, "get_Kondo_hamiltonian", "getEtaKondo"
                                                  )
        computation_results = wfrg.computations(decomposition_arr,
                                                {"VNE": [2, 3],
                                                }
                                               )
        time_list.append(time() - t)
        # plt.plot(computation_results["VNE"], 'o')
        # plt.show()
        # plt.plot(computation_results["VNE"], 'o')
    print (time_list)

KondoMERG(0.5, 2, True)

In [ ]:
t1 = [0.02578449249267578, 0.03951835632324219, 0.05424642562866211, 0.06536149978637695, 0.05416464805603027, 0.11724495887756348, 0.1581580638885498, 0.1907508373260498, 0.23258066177368164, 0.2877771854400635, 0.34641194343566895, 0.4238739013671875, 0.5073325634002686, 0.6056020259857178, 0.7212538719177246, 0.8474161624908447, 0.9987595081329346, 1.1645879745483398, 1.3628499507904053, 1.5197420120239258, 1.7492899894714355, 2.011781692504883, 2.321974277496338, 2.5959856510162354]
t2 = [0.015064001083374023, 0.013162851333618164, 0.017250776290893555, 0.024877309799194336, 0.036719322204589844, 0.053192853927612305, 0.07523965835571289, 0.10435986518859863, 0.13912582397460938, 0.18130946159362793, 0.2343451976776123, 0.2945723533630371, 0.3635547161102295, 0.4443998336791992, 0.5391242504119873, 0.649742603302002, 0.7718939781188965, 0.9061899185180664, 1.0617239475250244, 1.2393484115600586, 1.4299988746643066, 1.718707799911499, 1.8764243125915527, 2.137733221054077]
plt.plot(t1, label=r"old")
plt.plot(t2, label=r"new")
plt.legend()

In [ ]:
len([])

In [ ]:
basis = wfrg.get_basis(4)

In [ ]:
x = dict([['a', 1], ['b', 2]])

In [ ]:
[i for i in x.values()]

In [ ]:
_ = wfrg.get_computational_coefficients(basis, [0,1,1,0])


In [ ]:
print (wfrg.visualise_state(basis, [0,1,-1,0]))

In [ ]:
from collections import defaultdict

my_dict = defaultdict(float)
my_dict["a"] = 1

In [ ]:
my_dict["b"] += 1

In [ ]:
x = bytearray("10", encoding='ascii')

In [ ]:
from operator import itemgetter

In [ ]:
a = "abcd"
"".join(itemgetter(*(1,3))(a))